## The Fixed point method
__MATH 420__ <br>
_Spring 2021_ <br>

Here is a simple implementation of the fixed point method. Let's default the tolerance to 64 times the machine epsilon of the number with type x. And we'll default the maximum number of iterations to 50

In [ ]:
function fixed_point(f::Function, x::Number, tol::Number = 64*eps(typeof(x)), maxiter::Integer = 50)
    not_done = true
    xx = f(x)
    while !isapprox(x, xx, atol=tol, rtol=tol) && !isnan(x) && !isinf(x) && maxiter > 0
        @show(x)
        maxiter -= 1
        x = xx
        xx = f(x)
    end
    maxiter > 0 || error("Fixed point sequence converges too slowly.")
    x
end;

Let's find the fixed points  of $x \mapsto  x^2 - 5 x + 4$ are 1 and 4. We'll start with a graphical analysis

In [ ]:
using Gadfly, ForwardDiff

In [ ]:
F = x -> x^2-4*x + 4

In [ ]:
plot([x -> x, F],-1,6)

OK, it looks like the fixed points are $1$ and $4$. We can check this

In [ ]:
F(1), F(4)

The value of the derivative at each fixed point tells us we expect the fixed point sequence to converge; let's check:

In [ ]:
dF = x -> ForwardDiff.derivative(F,x)

In [ ]:
(dF(1), dF(4))

Both derivatives have magnitude greater than $1$. So we expect the fixed point sequences to diverge.

Try a starting value of $1.01$. Although this is super close to one fixed point, the fixed point sequence wanders around and doesn't appear to converge. At least initially, the terms of the sequence oscillate between being above the fixed point 1 to being below. This is due to the fact that $F^\prime(1) < 0$.

In [ ]:
fixed_point(F, 1.001)

Again since $F^\prime (1) = -2$, so we don't expect the fixed point sequence to converge; it's worse t 4, where the derivative at that fixed point is $4$.  Since $F^\prime(4) > 0$, we expect the sequence to diverge to infinity. With a starting value of  $4.0000001$, it does.

In [ ]:
fixed_point(F, 4.0000001)

One cure is to algebraically transform the equation $ x^2 - 4 x + 4 = x$ into an equivalent fixed point problem. There are lots of ways to do this. One way is to solving for $x^2$. Thus
$$
x^2 = 5 x - 4.
$$
Dividing by $x$ gives a fixed point problem
$
 x = \frac{5 x -4}{x}.
$
Of course if zero were a fixed point (it isn't), the division by zero would have eliminated zero as a fixed point.
Let's define $G(x) = \frac{5 x -4}{x}$ and look at its fixed point sequences

In [ ]:
G = x ->  (5*x-4)/x

Study this graphically--the slope a the fixed point looks to have a magnitude larger than 1 (divergence), but the slope at the fixed point $4$ looks shallow (convergence).

In [ ]:
plot([x -> x, G],0.5,5)

In [ ]:
dG = x -> ForwardDiff.derivative(G,x)

Did we do our algebra correctly? Does $G$ have fixed points at $1$ and $4$. Yes!

In [ ]:
(G(1), G(4))

What about the derivatives of $G$ at the fixed points?

In [ ]:
dG(1), dG(4)

A fixed point sequence that starts near $4$ will converge; with a start near $1$, we don't expect it to converge to $1$ Oops--with a starting value of 1.5, the FP sequence converges to 4. Same with an initial value of 11.0

In [ ]:
fixed_point(G, 1.5)

In [ ]:
fixed_point(G, 11.0)

### The Wegstein method;

From $F(x)$  we subtract $q (F(x) - x)$, where we attempt to choose the number $q$ so that the derivative of $x \mapsto F(x) + q (F(x) - x)$ vanishes at the fixed point.  Of course, we don't know the fixed point, so we'll use our initial guess as its approximate value.  For this choice, we have 
$q = -F^\prime(x_0) /(1 - F^\prime(x_0))$. The method is

In [ ]:
function fixed_pointX(f::Function, x::Number, tol::Number = eps(typeof(x)), maxiter::Integer = 125)
    df = x -> ForwardDiff.derivative(f,x)
    q = -df(x)/(1-df(x))
    fixed_point(x -> (1-q)*f(x) + q*x,x,tol,maxiter)
end

Returning to the function $F$, the Wegstein method allows for convergence to both fixed points!

In [ ]:
fixed_pointX(F, 1.5)

In [ ]:
fixed_pointX(F, 6.5)

In [ ]:
function wegstein(f::Function, x::Number, tol::Number = 64*eps(typeof(x)), maxiter::Integer = 25)
    not_done = true
    df = x -> ForwardDiff.derivative(f,x)
    function ff(x)
         q = -df(x)/(1-df(x))
        (1-q)*f(x) + q*x
    end
    fixed_point(ff, x, tol, maxiter)
    end;

In [ ]:
wegstein(F,1.3)

In [ ]:
wegstein(F,107.0)

Here is a version that returns an array of the terms of the fixed point sequence

In [ ]:
function fixed_point_collect(f::Function, x::Number, tol::Number = 64*eps(typeof(x)), maxiter::Integer = 50)
    not_done = true
    xx = f(x)
    p = [x]
    while !isapprox(x, xx, atol=tol, rtol=tol) && !isnan(x) && !isinf(x) && maxiter > 0
        push!(p,x)
        maxiter -= 1
        x = xx
        xx = f(x)
    end
    maxiter > 0 || error("Fixed point sequence converges too slowly.")
    p
end;

In [ ]:
p = fixed_point_collect(G, 53.0,1.0e-20)

For linear convergence of a fixed point sequence we expect
$$
  \lim_{k \to \infty} \frac{x_{k+2} - x_{k+1}}{x_{k+1} - x_{k}} = F\prime(x^\star)
$$
Let's check--recall that $G^\prime(4) = 1/4$. 

In [ ]:
Rate = n -> (p[n+2] - p[n+1])/(p[n+1] - p[n])

In [ ]:
xx = [k for k=1:length(p)-2];

In [ ]:
yy = map(Rate, xx)

In [ ]:
plot(x=xx, y=yy)